In [ ]:
# Read in libraries
# Specify directory for geo data
# Specify directory for processed data
library(dplyr)
library(stringr)
library(foreach)
library(jsonlite)
library(purrr)
library(tidyr)
library(readr)
library(stringr)
geo_data_directory <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/safe_graph_geos/2020/'
processed_data_directory <- '/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/'

In [ ]:
# Get list of months
months <- list.files(geo_data_directory)

# Iterate through months
data_wrapped <- foreach(i = months, .combine='rbind') %do% {
    directory = str_c(geo_data_directory, i)
    
    # How many days in each month?
    n_days = case_when(i == '01' ~ 31,
                       i == '02' ~ 29,
                       i == '03' ~ 31,
                       i == '04' ~ 19)
    
    # Loop through dates
    data_aggregate_month <- foreach(j = seq(1, n_days, 1), .combine='rbind') %do% {
        # Generate filename
        day_string <- str_pad(as.character(j), 2, '0', side='left')
        file = str_c(directory, '/2020', as.character(i), as.character(day_string), '_county_geos.txt')
        
        # Read in file
        data = read_tsv(file)
        
        # Name the columns
        names(data) <- c('origin_county', 'ds', 'device_count', 'non_home_cbg_visits_within_county',
                'cbg_visits_outside_county', 'home_cbg_visits', 'outside_device_county_visits')
        data.frame(data)
    }
}

In [ ]:
# Filter out origin_counties that are not in the 50 states
data_wrapped %>% 
  filter(origin_county <= 57000) -> county_mobility_dvs

In [ ]:
# Write to file
save(county_mobility_dvs, file=paste0(c(processed_data_directory,
                                         'county_mobility_dvs.Rdata'),
                                       sep='',
                                       collapse=''))